In [3]:
from IPython.core.interactiveshell import InteractiveShell # type: ignore
InteractiveShell.ast_node_interactivity='all'
from ML import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("GPU is","avaliable" if tf.config.experimental.list_physical_devices("GPU") else "not available")

GPU is not available


In [5]:
'''
We first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.

We then develop a given leaner (i.e., a phishing detector) by using the train data and a Convolutional Neural Network algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).

We then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.
'''

'\nWe first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.\n\nWe then develop a given leaner (i.e., a phishing detector) by using the train data and a Convolutional Neural Network algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).\n\nWe then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.\n'

In [6]:
# load data and split test and train data
json_file='../data_folder/deltaphish/preprocessed/normal/deltaphish_total_feature_modified.json'
data_delta=data_load(json_file)


# remove evasion dataset, then build new PWDs using remain data
phish_sub_test_x=joblib.load('../data_folder/deltaphish/preprocessed/phish_sub_test_x_100.pkl')
#print('phish_test_x index',phish_sub_test_x.index)#100
#print('phish_sub_test_x shape',phish_sub_test_x.shape)


data_delta_change=data_delta.copy()


#get the 100 phish samples and add them to the test dataset
phish_sub_test_later=data_delta_change.loc[phish_sub_test_x.index].copy()
data_delta_change=data_delta_change.drop(phish_sub_test_x.index)# del those 100 samples

train_random,test_random=train_test_split(data_delta_change,train_size=3200,test_size=700,shuffle=True,random_state=501)
train_x0=train_random.iloc[:,0:58]
train_y0=train_random.iloc[:,58]

#test_later=pd.concat(test_random,phish_sub_test_later)
frames =[test_random,phish_sub_test_later]
test_later=pd.concat(frames)
test_x0=test_later.iloc[:,0:58] 
test_y0=test_later.iloc[:,58] 

#use only html features
train_html_x0=train_random.iloc[:,27:49]
test_html_x0=test_later.iloc[:,27:49]

#only url features
train_url_x0=pd.concat([train_random.iloc[:,0:27],train_random.iloc[:,49:58]],axis=1)
test_url_x0=pd.concat([test_later.iloc[:,0:27],test_later.iloc[:,49:58]],axis=1)
train_y1=train_url_y0=train_html_y0=train_y0
test_y1=test_url_y0=test_html_y0=test_y0

#benign data 
benign_index=test_y0[test_y0==0].index.tolist()#
phish_index=test_y0[test_y0==1].index.tolist()#

#generate benign_test_x_y
benign_test_x=test_x0.loc[benign_index,:]
benign_test_y= test_y0.loc[benign_index]# 
benign_test_url_x=pd.concat([benign_test_x.iloc[:,0:27],benign_test_x.iloc[:,49:58]],axis=1)
benign_test_html_x=benign_test_x.iloc[:,27:49]
#100 phishing data already selected from test data to calculate the baseline recall
phish_sub_test_y=test_y0[phish_sub_test_x.index]#100 phishing samples
phish_sub_test_html_x=phish_sub_test_x.iloc[:,27:49]
phish_sub_test_url_x=pd.concat([phish_sub_test_x.iloc[:,0:27],phish_sub_test_x.iloc[:,49:58]],axis=1)

# data format for cnn
one_hot_train_y0=tf.one_hot(indices=train_y0.array,depth=2,on_value=1.0,off_value=0.0)
one_hot_test_y0=tf.one_hot(indices=test_y0.array,depth=2,on_value=1.0,off_value=0.0)
test_x_re0=np.asarray(test_x0).reshape(test_x0.shape[0],test_x0.shape[1],1)
train_x_re0=np.asarray(train_x0).reshape(train_x0.shape[0],train_x0.shape[1],1)

train_html_x_re0=np.asarray(train_html_x0).reshape(train_html_x0.shape[0],train_html_x0.shape[1],1)
test_html_x_re0=np.asarray(test_html_x0).reshape(test_html_x0.shape[0],test_html_x0.shape[1],1)
train_url_x_re0=np.asarray(train_url_x0).reshape(train_url_x0.shape[0],train_url_x0.shape[1],1)
test_url_x_re0=np.asarray(test_url_x0).reshape(test_url_x0.shape[0],test_url_x0.shape[1],1)

benign_test_x_re=np.asarray(benign_test_x).reshape(benign_test_x.shape[0],benign_test_x.shape[1],1)
benign_test_html_x_re=np.asarray(benign_test_html_x).reshape(benign_test_html_x.shape[0],benign_test_html_x.shape[1],1)
benign_test_url_x_re=np.asarray(benign_test_url_x).reshape(benign_test_url_x.shape[0],benign_test_url_x.shape[1],1)


phish_sub_test_x_re=np.asarray(phish_sub_test_x).reshape(phish_sub_test_x.shape[0],phish_sub_test_x.shape[1],1)
phish_sub_test_html_x_re=np.asarray(phish_sub_test_html_x).reshape(phish_sub_test_html_x.shape[0],phish_sub_test_html_x.shape[1],1)
phish_sub_test_url_x_re=np.asarray(phish_sub_test_url_x).reshape(phish_sub_test_url_x.shape[0],phish_sub_test_url_x.shape[1],1)



2024-06-21 15:24:20.807198: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# training

In [7]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)


model_cnn_combined=create_model_cnn()
model_cnn_url=create_model_url()
model_cnn_html=create_model_html()

print('***************fit cnn_combined model:')
model_cnn_combined.fit(train_x_re0,one_hot_train_y0,batch_size=32,epochs=200, verbose=2)
print('***************fit cnn_url model:')
model_cnn_url.fit(train_url_x_re0,one_hot_train_y0,batch_size=32,epochs=200, verbose=2)
print('***************fit cnn_html model:')
model_cnn_html.fit(train_html_x_re0,one_hot_train_y0,batch_size=32,epochs=200, verbose=2)

***************fit cnn_combined model:
Epoch 1/200
100/100 - 1s - loss: 0.4033 - categorical_accuracy: 0.8637 - 959ms/epoch - 10ms/step
Epoch 2/200
100/100 - 0s - loss: 0.3214 - categorical_accuracy: 0.8741 - 362ms/epoch - 4ms/step
Epoch 3/200
100/100 - 0s - loss: 0.2585 - categorical_accuracy: 0.8969 - 373ms/epoch - 4ms/step
Epoch 4/200
100/100 - 0s - loss: 0.2075 - categorical_accuracy: 0.9209 - 368ms/epoch - 4ms/step
Epoch 5/200
100/100 - 0s - loss: 0.1838 - categorical_accuracy: 0.9334 - 370ms/epoch - 4ms/step
Epoch 6/200
100/100 - 0s - loss: 0.1678 - categorical_accuracy: 0.9378 - 356ms/epoch - 4ms/step
Epoch 7/200
100/100 - 0s - loss: 0.1577 - categorical_accuracy: 0.9453 - 361ms/epoch - 4ms/step
Epoch 8/200
100/100 - 0s - loss: 0.1518 - categorical_accuracy: 0.9463 - 356ms/epoch - 4ms/step
Epoch 9/200
100/100 - 0s - loss: 0.1497 - categorical_accuracy: 0.9488 - 367ms/epoch - 4ms/step
Epoch 10/200
100/100 - 0s - loss: 0.1447 - categorical_accuracy: 0.9522 - 371ms/epoch - 4ms/step

***************fit cnn_url model:
Epoch 1/200
100/100 - 1s - loss: 0.3962 - categorical_accuracy: 0.8619 - 1s/epoch - 13ms/step
Epoch 2/200
100/100 - 1s - loss: 0.3408 - categorical_accuracy: 0.8700 - 903ms/epoch - 9ms/step
Epoch 3/200
100/100 - 1s - loss: 0.3337 - categorical_accuracy: 0.8725 - 920ms/epoch - 9ms/step
Epoch 4/200
100/100 - 1s - loss: 0.3388 - categorical_accuracy: 0.8716 - 889ms/epoch - 9ms/step
Epoch 5/200
100/100 - 1s - loss: 0.3243 - categorical_accuracy: 0.8800 - 894ms/epoch - 9ms/step
Epoch 6/200
100/100 - 1s - loss: 0.3205 - categorical_accuracy: 0.8784 - 888ms/epoch - 9ms/step
Epoch 7/200
100/100 - 1s - loss: 0.3228 - categorical_accuracy: 0.8734 - 886ms/epoch - 9ms/step
Epoch 8/200
100/100 - 1s - loss: 0.3174 - categorical_accuracy: 0.8806 - 868ms/epoch - 9ms/step
Epoch 9/200
100/100 - 1s - loss: 0.3147 - categorical_accuracy: 0.8822 - 872ms/epoch - 9ms/step
Epoch 10/200
100/100 - 1s - loss: 0.3105 - categorical_accuracy: 0.8838 - 859ms/epoch - 9ms/step
Epoch 1

***************fit cnn_html model:
Epoch 1/200
100/100 - 1s - loss: 0.3786 - categorical_accuracy: 0.8637 - 704ms/epoch - 7ms/step
Epoch 2/200
100/100 - 0s - loss: 0.2884 - categorical_accuracy: 0.8772 - 222ms/epoch - 2ms/step
Epoch 3/200
100/100 - 0s - loss: 0.2495 - categorical_accuracy: 0.9187 - 239ms/epoch - 2ms/step
Epoch 4/200
100/100 - 0s - loss: 0.2213 - categorical_accuracy: 0.9297 - 248ms/epoch - 2ms/step
Epoch 5/200
100/100 - 0s - loss: 0.2077 - categorical_accuracy: 0.9337 - 246ms/epoch - 2ms/step
Epoch 6/200
100/100 - 0s - loss: 0.1951 - categorical_accuracy: 0.9362 - 248ms/epoch - 2ms/step
Epoch 7/200
100/100 - 0s - loss: 0.1939 - categorical_accuracy: 0.9409 - 246ms/epoch - 2ms/step
Epoch 8/200
100/100 - 0s - loss: 0.1910 - categorical_accuracy: 0.9409 - 246ms/epoch - 2ms/step
Epoch 9/200
100/100 - 0s - loss: 0.1810 - categorical_accuracy: 0.9447 - 246ms/epoch - 2ms/step
Epoch 10/200
100/100 - 0s - loss: 0.1800 - categorical_accuracy: 0.9434 - 243ms/epoch - 2ms/step
Epoc

# Baseline Performance- refer to Table 4, top row. No Attack and no Evasion space involved

In [8]:

# The get_sub_base_rf_recall function calculates the recall for each model using the selected features.
# It prints the recall values (TPR) for the combined (Fc), URL (Fu), and HTML (Fr) feature sets.
print("TPR Values")
print('Fc:')  
cnn_recall_com=get_base_recall_cnn(model_cnn_combined,test_x_re0,test_y0)
print('Fu:')  
cnn_recall_url=get_base_recall_cnn(model_cnn_url,test_url_x_re0,test_url_y0)
print('Fr:')  
cnn_recall_html=get_base_recall_cnn(model_cnn_html,test_html_x_re0,test_html_y0)


print("\n")
print('Fc:')    
cnn_fpr_com=get_base_fpr_cnn(model_cnn_combined,test_x_re0,test_y0)
print('Fu:')  
cnn_fpr_url=get_base_fpr_cnn(model_cnn_url,test_url_x_re0,test_url_y0)
print('Fr:')  
cnn_fpr_html=get_base_fpr_cnn(model_cnn_html,test_html_x_re0,test_html_y0)


TPR Values
Fc:
cnn_recall 0.93
Fu:
cnn_recall 0.51
Fr:
cnn_recall 0.80


Fc:
fpr 0.007
Fu:
fpr 0.035
Fr:
fpr 0.013


# No attack WA

In [9]:
print('cnn_no-atk_recall_combine:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_re,phish_sub_test_y)
print('cnn_no-atk_recall_url:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_re,phish_sub_test_y)
print('cnn_no-atk_recall_html:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_re,phish_sub_test_y)
 


cnn_no-atk_recall_combine:
cnn_recall 0.95
cnn_no-atk_recall_url:
cnn_recall 0.60
cnn_no-atk_recall_html:
cnn_recall 0.81


# wa_u attack

In [10]:
ad_url_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_u/phish_sub_problem_space_url_shorten100.json")
ad_url_problem_test_x=ad_url_problem_data.iloc[:,0:58]
ad_url_problem_test_y=ad_url_problem_data.iloc[:,58]

ad_url_problem_test_url_x=pd.concat([ad_url_problem_data.iloc[:,0:27],ad_url_problem_data.iloc[:,49:58]],axis=1)
ad_url_problem_test_html_x=ad_url_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_url_problem_test_x).reshape(ad_url_problem_test_x.shape[0],ad_url_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_url_problem_test_url_x).reshape(ad_url_problem_test_url_x.shape[0],ad_url_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_url_problem_test_html_x).reshape(ad_url_problem_test_html_x.shape[0],ad_url_problem_test_html_x.shape[1],1)

#...............
print('cnn_recall_combined_wa_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)



cnn_recall_combined_wa_u:
cnn_recall 0.90
cnn_recall_url_wa_u:
recall 1
cnn_recall_html_wa_u:
cnn_recall 0.80


# wa_r attack

In [11]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_r/phish_sub_problem_space_html_add50internallinks.json")#wa_u,wa_c
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)



cnn_recall_combined_wa_r:
cnn_recall 0.96
cnn_recall_url_wa_r:
cnn_recall 0.60
cnn_recall_html_wa_r:
cnn_recall 0.35


# wa_c attack

In [12]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_c/phish_sub_problem_space_html_add50internallinks_combined_shorten_url.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]
 
ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa_c:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa_c:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa_c:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)




cnn_recall_combined_wa_c:
cnn_recall 0.91
cnn_recall_url_wa_c:
recall 1
cnn_recall_html_wa_c:
cnn_recall 0.34


# wa+_u attack

In [13]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_u/phish_sub_html_change_full_knowledge_change_again_later_only_url_shorten.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa+_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa+_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa+_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)



cnn_recall_combined_wa+_u:
cnn_recall 0.89
cnn_recall_url_wa+_u:
recall 1
cnn_recall_html_wa+_u:
cnn_recall 0.80


# wa+_r attack

In [14]:

ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_r/phish_sub_problem_space__full_onlyhtml_316.json")#wa+_u,wa+_c

ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa+_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa+_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa+_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)


cnn_recall_combined_wa+_r:
cnn_recall 0.89
cnn_recall_url_wa+_r:
cnn_recall 0.60
cnn_recall_html_wa+_r:
cnn_recall 0.85


# wa+_c attack

In [15]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_c/phish_sub_problem_space_mix_url_html_316.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa+_c:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa+_c:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa+_c:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)


cnn_recall_combined_wa+_c:
cnn_recall 0.86
cnn_recall_url_wa+_c:
recall 1
cnn_recall_html_wa+_c:
cnn_recall 0.87


# pa_u attack

In [16]:

phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_u.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_pa_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_pa_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_pa_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)



cnn_recall_combined_pa_u:
cnn_recall 0.99
cnn_recall_url_pa_u:
cnn_recall 0.65
cnn_recall_html_pa_u:
cnn_recall 0.49


# pa_r attack

In [17]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_r.pkl")

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_pa_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_pa_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_pa_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)




cnn_recall_combined_pa_r:
cnn_recall 0.63
cnn_recall_url_pa_r:
cnn_recall 0.60
cnn_recall_html_pa_r:
cnn_recall 0.56


# pa_c attack

In [18]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_c.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_pa_c:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_pa_c:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_pa_c:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)



cnn_recall_combined_pa_c:
cnn_recall 0.99
cnn_recall_url_pa_c:
cnn_recall 0.65
cnn_recall_html_pa_c:
cnn_recall 0.49


# ma_u attack

In [19]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1
phish_sub_test_x_ch['URL_redirect']=-1
phish_sub_test_x_ch['URL_shortener']=1
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 
phish_sub_test_x_ch['URL_checkPathExtend']=-1
phish_sub_test_x_ch['URL_checkPunycode']=-1
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0

phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
phish_sub_test_x_ch['REP_checkGI']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_ma_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_ma_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_ma_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)



cnn_recall_combined_ma_u:
cnn_recall 0.00
cnn_recall_url_ma_u:
cnn_recall 0.00
cnn_recall_html_ma_u:
cnn_recall 0.81


# ma_r attack

In [20]:
phish_sub_test_x_ch=phish_sub_test_x.copy()

phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_ma_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_ma_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_ma_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)


cnn_recall_combined_ma_r:
cnn_recall 0.00
cnn_recall_url_ma_r:
cnn_recall 0.60
cnn_recall_html_ma_r:
cnn_recall 0.00


# ma_c attack

In [21]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1#single no change
phish_sub_test_x_ch['URL_redirect']=-1#
phish_sub_test_x_ch['URL_shortener']=1 #single no change
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 #single no change
phish_sub_test_x_ch['URL_checkPathExtend']=-1#single no change
phish_sub_test_x_ch['URL_checkPunycode']=-1#single no change
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0

phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0 
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
 
phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
 
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1#1#0
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1
phish_sub_test_x_ch['REP_checkGI']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_ma_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_ma_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_ma_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)


cnn_recall_combined_ma_u:
cnn_recall 0.00
cnn_recall_url_ma_u:
cnn_recall 0.00
cnn_recall_html_ma_u:
cnn_recall 0.00
